In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import json
import threading
from datetime import datetime, timedelta



In [55]:
def get_oup_extra_data(url):
    driver = webdriver.Chrome()
    driver.get(url)
    html = driver.page_source
    driver.close()
    soup=BeautifulSoup(html,'html.parser')
    try:
        abstract = soup.find('section', class_='abstract').text
    except:
        try:
            h2_lst = soup.find_all('h2', class_='section-title js-splitscreen-section-title')
            limited_lst = []
            for i in range(len((h2_lst))):
                if 'intro' in h2_lst[i].text.lower() or 'abstract' in h2_lst[i].text.lower():
                    limited_lst.extend([h2_lst[i]])
            abstract = ""
            for a in limited_lst[0].find_next_siblings():
                if 'section-title js-splitscreen-section-title' in str(a):
                    break
                abstract+=a.text+' '
        except:
            abstract = ""
    script_tag = soup.find('script', type='application/ld+json')
    script_content = script_tag.string
    data = json.loads(script_content)
    keywords = data.get("keywords", [])
    authors = []
    affiliations = []
    try:
        for name in data['author']:
            authors.append(name['name'])
            affiliations.append(name['affiliation'])
        affiliations= list(set(affiliations))
        authors= list(set(authors))
    except:
        try:
            author_meta_tags = soup.find_all('meta', {'name': 'citation_author'})
            authors = []
            for auth in author_meta_tags:
                authors.append(auth['content'])
            authors =list(set(authors))
            institution_meta_tags = soup.find_all('meta', {'name': 'citation_author_institution'})
            affiliations = []
            for auth in institution_meta_tags:
                affiliations.append(auth['content'])
            affiliations =list(set(affiliations))
        except:
            pass
    return abstract, keywords, authors, affiliations


def get_oup_data(div):
    try:
        global out
        paper_info = {}
        paper_info['title'] = div.find('a', class_='article-link at-sr-article-title-link').text.strip()
        paper_info['date']=datetime.strptime(div.find('div', class_='sri-date al-pub-date').text.strip().split(': ')[-1],'%d %B %Y' ).strftime("%Y-%m-%d")
        url = div.find('a', class_="article-link at-sr-article-title-link")['href']
        paper_info['paper_id']= url[url.find('/doi'):url.find('?search')]
        paper_info['url'] = 'https://academic.oup.com'+url
        paper_info['abstract'], paper_info['subjects'], paper_info['authors'], paper_info['affiliations'] =  get_oup_extra_data(paper_info['url'])
        paper_info['source'] = 'Oxford Academic'
        out.append(paper_info)
    except:
        print('fail')
        pass

In [60]:
out = []
def get_oup_days_back(num_days):
    global out
    out= []
    i=1
    while True:
        url = f'https://academic.oup.com/journals/search-results?sort=Date+%E2%80%93+Newest+First&f_ContentSubTypeDisplayName=Research+Article&fl_SiteID=5567&page={i}'
        driver = webdriver.Chrome()
        driver.get(url)
        html = driver.page_source
        driver.close()
        soup=BeautifulSoup(html,'html.parser')
        threads = []
        for div in soup.find_all('div', class_='sr-list al-article-box al-normal clearfix'):
            thread = threading.Thread(target=get_oup_data, args=(div,))
            threads.append(thread)
            thread.start()
        for thread in threads:
            thread.join()
        i+=1
        if datetime.strptime(out[-1]['date'], '%Y-%m-%d').date() <datetime.now().date()-timedelta(days=num_days):
            break
    return pd.DataFrame.from_records(out)
df=get_oup_days_back(2)


fail
fail


,title,date,paper_id,url,abstract,subjects,authors,affiliations,source
0,Should we use rifampicin in periprosthetic joi...,2023-10-06,/doi/10.1093/ofid/ofad491/7293986,https://academic.oup.com/ofid/advance-article/...,BackgroundPrevious studies demonstrated the ef...,[],"[Soriano, Alex, Kramer, Tobias Siegfried, Tatt...","[Infectious Diseases and Intensive Care Unit, ...",Oxford Academic
1,N-Terminal pro-Brain Natriuretic Peptide is a ...,2023-10-06,/doi/10.1093/rheumatology/kead522/7295625,https://academic.oup.com/rheumatology/advance-...,ObjectivesPrediction models based on tradition...,"[Systemic Lupus Erythematosus, Brain Natriuret...","[Bernatsky, Sasha, Pineau, Christian A, Kalach...","[Division of Rheumatology, McGill University ...",Oxford Academic
2,Ovarian follicle size or growth-rate can both ...,2023-10-06,/doi/10.1093/biolre/ioad134/7295315,https://academic.oup.com/biolreprod/advance-ar...,The endocrinology that leads to the ovulation ...,"[Poly-ovulatory, Graafian follicle, Folliculog...","[Richard, Sharon, Jasoni, Christine L, Zhou, Y...","[Department of Anatomy , School of Biomedical...",Oxford Academic
3,Crohn’s Disease of the Ileoanal Pouch: A High ...,2023-10-06,/doi/10.1093/ibd/izad228/7295304,https://academic.oup.com/ibdjournal/advance-ar...,BackgroundApproximately 1 in 10 patients with ...,"[ileal pouch–anal anastomosis, ulcerative coli...","[Wessels, Elise, Buskens, Christianne, Goetgeb...",[Department of Gastroenterology and Hepatology...,Oxford Academic
4,Reconciling paid work and informal caregiving ...,2023-10-06,/doi/10.1093/geroni/igad115/7293241,https://academic.oup.com/innovateage/advance-a...,Background and ObjectivesMany older adults who...,"[Informal Caregiving, employment, older adults]","[Koreshi, Shanika Yoshini, Alpass, Fiona]","[School of Psychology, Massey University , Man...",Oxford Academic
...,...,...,...,...,...,...,...,...,...
333,Computer image analysis with artificial intell...,2023-10-04,/doi/10.1093/postmj/qgad095/7289070,https://academic.oup.com/pmj/advance-article/d...,"Artificial intelligence tools, particularly co...","[biotechnology & bioinformatics, education and...","[Hunter, James, Thomson, Brian, Kourounis, Geo...","[Nuffield Department of Surgical Sciences, Un...",Oxford Academic
334,How can UK public health initiatives support e...,2023-10-04,,https://academic.oup.com/heapro/article/38/5/d...,Physical activity improves physical and mental...,"[physical activity, running, participation, pa...","[Relph, Nicola, Owen, Michael, Noonan, Rob, Bu...","[The Advanced Wellbeing Research Centre, Sheff...",Oxford Academic
335,Adolescent Cybervictimization in 31 Countries:...,2023-10-03,/doi/10.1093/socpro/spad049/7288199,https://academic.oup.com/socpro/advance-articl...,"This study examined the gender gap, gendered o...","[gender inequality, cybervictimization, backla...","[Wilcox, Pamela, Deryol, Rustu, McNealey, Rach...","[Pennsylvania State University , USA, Univers...",Oxford Academic
336,"The family meal, a ritual frozen in time; an A...",2023-10-04,,https://academic.oup.com/heapro/article/38/5/d...,Family meals are recognized as an opportunity ...,"[family meal, grounded theory, food work, food...","[Coveney, John, Middleton, Georgia, Golley, Re...","[College of Nursing and Health Sciences, Flind...",Oxford Academic


In [59]:
df[df['affiliations'].astype(str).str.contains('Georgia')]

,title,date,paper_id,url,abstract,subjects,authors,affiliations,source
137,Protein dose-sparing effect of AS01B adjuvant ...,2023-10-05,/doi/10.1093/infdis/jiad434/7289550,https://academic.oup.com/jid/advance-article/d...,BackgroundHVTN 120 is a phase 1/2a randomized ...,"[HIV, vaccine, dose, adjuvant]","[Corey, Lawrence, McElrath, M Juliana, Chirenj...","[ZMC University of California San Francisco, D...",Oxford Academic


In [26]:
a = get_oup_extra_data('https://academic.oup.com/bjsw/advance-article/doi/10.1093/bjsw/bcad217/7295480?searchresult=1')
a

('A continued and unprecedented demand for England’s child protection services has coincided with increasing numbers of children being made subject to Care Orders within the Family Court. There is a growing understanding of the associated challenges—not least in terms of cost and placement availability. However, there has been limited discussion of the difficulties associated with children being made subject to a Care Order, but who remain living at home with their parents. This article reports on an iterative mixed method study which sought to explore child protection social workers’ experiences of ‘Care Orders at home’. This article reports on the prevalence of ‘Care Orders at home’ within Northwest England and identifies specific challenges in the context of ‘managing child protection partners’ expectations’; apparent ‘confusion and disempowerment of the child and family’ and ‘an increased sense of risk’ with respect to several of the key stakeholders involved. Implications that eme

In [28]:
len(a)

4

In [48]:
out

[]

In [52]:
date_txt = out[-1]['date']
date_txt

'2023-10-06'

In [53]:
datetime.strptime(out[-1]['date'], '%Y-%m-%d').date()

datetime.date(2023, 10, 6)

["Department of Pediatrics, University of Cincinnati College of Medicine, Cincinnati Children's Hospital Medical Center, Cincinnati, Ohio, USA",
 "Department of Infectious Diseases, Children's, St. Jude Children's Research Hospital, Memphis, Tennessee, USA",
 "Department of Pediatrics, University of Washington and Seattle Children's Research Institute, Seattle, Washington, USA",
 "Department of Pediatrics, Division of Infectious Diseases & Host Defense, Nationwide Children's Hospital and The Ohio State University, Columbus, Ohio, USA",
 'Department of Pediatrics, Vanderbilt University Medical Center, Nashville, Tennessee, USA',
 'Department of Biostatistics, Vanderbilt University Medical Center, Nashville, Tennessee, USA',
 "Department of Pediatrics, Division of Infectious Diseases, Baylor College of Medicine, and Texas Children's Hospital, Houston, Texas, USA",
 "Department of Pediatrics, University of California San Francisco and Benioff Children's Hospital – San Francisco, San Franc

In [92]:
h2_lst = soup.find_all('h2', class_='section-title js-splitscreen-section-title')
limited_lst = []
for i in range(len((h2_lst))):
    if 'intro' in h2_lst[i].text.lower() or 'abstract' in h2_lst[i].text.lower():
        limited_lst.extend([h2_lst[i]])
test_abstract = ""
for a in limited_lst[0].find_next_siblings():
    if 'section-title js-splitscreen-section-title' in str(a):
        break
    test_abstract+=a.text+' '
test_abstract

'The current investment treaty system is, by its nature, highly fragmented, and there is a large potential for conflict, within the system and with other fields of public international law. Several States have already been exposed to conflicts within the investment treaty system, as well as collisions with other treaty regimes. The CME v Czech Republic and Lauder v Czech Republic cases2 are the prime examples— also referred to as the ‘ultimate fiasco in investment arbitration’3—of investment arbitrations resulting in directly contradictory outcomes. While the Lauder v Czech Republic Tribunal denied all claims for damages by the ultimate shareholder of a Czech television broadcasting company, the CME v Czech Republic Tribunal, 10 days later, awarded the Dutch investment vehicle compensation for several treaty violations based on the same treatment of the same investment and applying essentially the same investment treaty standards.4 EU Member States have also experienced clashes between